In [168]:
import math
import os
from pathlib import Path
import pandas as pd
import re
from datetime import datetime
import math

# Load the required libraries
from rdflib import Graph, Literal, RDF, URIRef, Namespace

# RDFLib knows about some namespaces, like FOAF and XSD
from rdflib.namespace import FOAF, XSD

In [169]:
absPath = str(Path(os.path.abspath(os.getcwd())).absolute())
datasetsPath = os.path.join(absPath, "cleaned_datasets")
rdfPath = os.path.join(absPath, "rdf")
print(absPath)
print(datasetsPath)
print(rdfPath)

games = os.path.join(datasetsPath, "cleaned_games.csv")
platforms = os.path.join(datasetsPath, "platforms.csv")
print(platforms)

C:\Users\Daniel\Desktop\DB2 Git\HLTB-unipd
C:\Users\Daniel\Desktop\DB2 Git\HLTB-unipd\cleaned_datasets
C:\Users\Daniel\Desktop\DB2 Git\HLTB-unipd\rdf
C:\Users\Daniel\Desktop\DB2 Git\HLTB-unipd\cleaned_datasets\platforms.csv


In [170]:
# Country Ontology
CNS = Namespace("http://eulersharp.sourceforge.net/2003/03swap/countries#")

# HLTB Ontology
GA = Namespace("https://www.dei.unipd.it/game#")


##GENRE

In [171]:
def createGraph():
    # Create the graph
    g = Graph()

    # Bind the namespaces to a prefix for more readable output
    g.bind("xsd", XSD)
    g.bind("countries", CNS)
    g.bind("ga", GA)

    return g

In [172]:
# Create Graph
g = createGraph()
# Load the CSV files in memory
genres = pd.read_csv(games, sep=",", index_col="genres")
genres.to_csv("prova.txt")


In [173]:



def setGenreID(genre):
    genre = str(genre).replace("/",", ").replace("nan","")
    genre=genre.split(", ")
    list=[]
    for i in range(len(genre)):
        list.append([])
        list[i].append(genre[i])
        list[i].append(genre[i].lower().replace("'","").replace(" ", "-"))
    return(list)

aa=[]

for genre, row in genres.iterrows():
    allGenres = setGenreID(genre)
    for iterator in allGenres:
        aa.append(iterator[1])
        if not (iterator[0] == " " or iterator[0] == ""):
            Genre = URIRef(GA[iterator[1]])
            #Add triples using store's add() method.
            g.add((Genre, RDF.type, GA.Genre))
            # Add the name of the genre
            g.add((Genre, GA["name"], Literal(iterator[0], datatype=XSD.string)))



In [174]:
# Save data in the Turtle format
with open("prova.txt", "w", encoding="utf-8") as fp:
    fp.write(g.serialize(format="turtle"))

#PLATFORM

In [175]:
# Create Graph
g = createGraph()
# Load the CSV files in memory
platforms = pd.read_csv(platforms, sep=",", index_col="Platform")
platforms.to_csv("prova.txt")

In [176]:
def setPlatformID(platform):
    return(platform.lower().replace(" ", "-"))

In [177]:
for platform, row in platforms.iterrows():
    Platform = URIRef(GA[setPlatformID(platform)])
    #Add triples using store's add() method.
    g.add((Platform, RDF.type, GA.Platform))
    # Add the name of the genre
    g.add((Platform, GA["name"], Literal(platform, datatype=XSD.string)))

    #Add popularity if platform is popular
    if(row["Popular"] == True):
        g.add((Platform, GA["popular"], Literal(True, datatype=XSD.boolean)))

    #Add release date if present
    if pd.notna(row["Release date"]):
        time = datetime.combine(datetime.strptime(row["Release date"], '%Y-%M-%d'), datetime.min.time())
        g.add((Platform, GA["releaseDate"], Literal(time,datatype=XSD.dateTime)))

    #Add CPU information
    if pd.notna(row["CPU"]):
        g.add((Platform, GA["cpu"], Literal(row["CPU"],datatype=XSD.string)))

    #Add CPU bit information
    if pd.notna(row["\"Bits\""]):
        bits = row["\"Bits\""].split("-")[0]
        g.add((Platform, GA["bits"], Literal(bits,datatype=XSD.int)))

    #Add acronym information
    if pd.notna(row["Acronym"]):
        g.add((Platform, GA["acronym"], Literal(row["Acronym"],datatype=XSD.string)))

In [178]:
# Save data in the Turtle format
with open("prova.txt", "w", encoding="utf-8") as fp:
    fp.write(g.serialize(format="turtle"))